In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [117]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_eg_year = pd.read_csv(path + "df_eg_year.csv")
df_eg_year_val = pd.read_csv(path + "df_eg_year_val.csv")
df_eg_year_test = pd.read_csv(path + "df_eg_year_test.csv")

In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm

def f_data(train, val, test):

    X_train = train.drop(['fr_yn'], 1)
    y_train = train['fr_yn']
    X_val = val.drop(['fr_yn'], 1)
    y_val = val['fr_yn']
    X_test = test.drop(['fr_yn'], 1)

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(train)]
    X_val = df_all[len(train):-len(test)]
    X_test = df_all[-len(test):]

    X_train = X_train.fillna(-1)
    X_val = X_val.fillna(-1)
    X_test = X_test.fillna(-1)

    # X_train = X_train.replace({-1:np.NaN})
    # X_val = X_val.replace({-1:np.NaN})
    # X_test = X_test.replace({-1:np.NaN})

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

def f1_rfc_2(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(random_state=42, n_estimators=1000)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
# def population(x):
#     if x < 1000:
#         return "1k"
#     elif x < 10000:
#         return "10k"
#     elif x < 100000:
#         return "100k"
#     elif x < 1000000:
#         return "1000k"

# df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
# df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
# df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

#concat eg_year
df_train_2 = pd.concat([df_train_2, df_eg_year], axis=1)
df_val_2 = pd.concat([df_val_2, df_eg_year_val], axis=1)
df_test_2 = pd.concat([df_test_2, df_eg_year_test], axis=1)

#for imputation
# s = df_train_2.isnull().sum()
# c_list = np.where(s >= max(s)*0.5)
# df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

# df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
# df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)

In [0]:
def preprocessing():
    path = "/content/drive/My Drive/fire_predict/base_data/"
    df_train = pd.read_csv(path + "PJT002_train.csv")
    df_val = pd.read_csv(path + "PJT002_validation.csv")
    df_test = pd.read_csv(path + "PJT002_test.csv")

    df_eg_year = pd.read_csv(path + "df_eg_year.csv")
    df_eg_year_val = pd.read_csv(path + "df_eg_year_val.csv")
    df_eg_year_test = pd.read_csv(path + "df_eg_year_test.csv")

    # lable -> binary
    binary_y = {'N': 0, 'Y': 1}

    df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
    df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

    #df ->df_2
    df_train_2 = df_train
    df_val_2 = df_val
    df_test_2 = df_test

    #drop id
    df_train_2 = df_train_2.drop("id", axis=1)
    df_val_2 = df_val_2.drop("id", axis=1)
    df_test_2 = df_test_2.drop("id", axis=1)

    #drop dt_of_fr
    df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
    df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
    df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

    #dt_of_athrztn -> year 

    df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
    df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
    df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
    df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
    df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
    df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

    # hm_cnt -> categorical
    # def population(x):
    #     if x < 1000:
    #         return "1k"
    #     elif x < 10000:
    #         return "10k"
    #     elif x < 100000:
    #         return "100k"
    #     elif x < 1000000:
    #         return "1000k"

    # df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
    # df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
    # df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

    #tmprtr c -> k
    df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
    df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
    df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

    #gas_engry_us_201507 x < 0 -> remove
    df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

    # drop gas, elec
    gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
    df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
    df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
    df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

    #concat eg_year
    df_train_2 = pd.concat([df_train_2, df_eg_year], axis=1)
    df_val_2 = pd.concat([df_val_2, df_eg_year_val], axis=1)
    df_test_2 = pd.concat([df_test_2, df_eg_year_test], axis=1)

    #for imputation
    # s = df_train_2.isnull().sum()
    # c_list = np.where(s >= max(s)*0.5)
    # df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

    # df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
    # df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

    # X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
    return df_train_2, df_val_2, df_test_2

In [30]:
f1_rfc(X_train, y_train, X_val, y_val)

100%|██████████| 10/10 [04:17<00:00, 25.70s/it]


0.5103954341622502

In [7]:
df_train_2.select_dtypes("number")

,fr_yn,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,e_14,e_15,e_16,e_17,e_18,e_t,g_14,g_15,g_16,g_17,g_18,g_t
0,1,3,69.420,69.4200,0.0,1977.0,1.0,0.0,283.85,NaN,0.9,200.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17360.0,4547,NaN,133,137.0,112,0,0,1891,29231,11322,88,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,3,46.290,46.2900,0.0,NaN,1.0,0.0,292.65,NaN,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1791.0,6388,122581.0,489,85.0,818,0,0,4533,20480,3369,7727,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,583.800,2516.7600,1446.0,2000.0,5.0,0.0,288.65,NaN,2.0,90.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17285.0,3340,618105.0,143,137.0,165,0,0,277,31197,12451,72,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,48.920,48.9200,0.0,1936.0,1.0,0.0,293.75,NaN,0.4,0.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7327.0,179,719542.0,1585,176.0,131,0,0,438,4270,407,508,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,0.000,0.0000,0.0,NaN,NaN,NaN,285.85,NaN,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17278.0,4822,NaN,603,137.0,686,0,0,1702,29778,12487,707,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,0,2,423.000,423.0000,3010.0,1996.0,NaN,NaN,284.35,NaN,0.2,0.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20955.0,3435,NaN,895,139.0,1131,0,0,3591,3624,6679,914,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59195,0,35,5593.711,54835.0382,0.0,NaN,171.0,0.0,287.95,NaN,1.9,20.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21014.0,1137,1849455.0,88,139.0,108,0,0,2629,77,1285,131,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59196,0,2,2436.820,2468.8000,6512.0,NaN,4.0,0.0,286.45,NaN,3.2,230.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3014.0,5875,NaN,25,139.0,968,2,0,4271,6139,4878,790,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59197,0,2,283.200,207.7100,0.0,2005.0,2.0,0.0,291.35,1.0,1.4,360.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3753.0,8579,NaN,510,139.0,177,0,0,1387,10142,3572,613,0,0.0,0.0,3869.0,6348.0,8084.0,20008.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_train_2.select_dtypes("object")

,bldng_us,bldng_archtctr,bldng_us_clssfctn,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,mlt_us_yn,trgt_crtr,fr_fghtng_fclt_spcl_css_5_yn,fr_fghtng_fclt_spcl_css_6_yn,us_yn,dngrs_thng_yn,slf_fr_brgd_yn,blk_dngrs_thng_mnfctr_yn,cltrl_hrtg_yn
0,단독주택,블록구조,주거용,대,자연녹지지역,지정되지않음,단독,세로한면(불),경상남도 창녕군 창녕읍,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,대,계획관리지역,지정되지않음,단독,세로한면(가),경상남도 의령군 가례면,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,공동주택,철근콘크리트구조,주거용,답,제2종일반주거지역,지정되지않음,연립,중로한면,경상남도 창녕군 창녕읍,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,단독주택,일반목구조,주거용,대,제1종일반주거지역,지정되지않음,단독,세로한면(불),경상남도 창원시의창구 대산면,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,임,자연환경보전지역,지정되지않음,자연림,세로한면(불),경상남도 창녕군 창녕읍,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,동.식물 관련시설,경량철골구조,NaN,목,계획관리지역,지정되지않음,주거기타,세로한면(가),경상남도 함안군 칠원읍,N,일반대상물,N,N,Y,N,N,N,N
59195,NaN,NaN,NaN,대,계획관리지역,지정되지않음,아파트,소로한면,경상남도 함안군 칠원읍,N,"스프링클러,물분무등설치대상",N,N,Y,N,N,N,N
59196,NaN,NaN,NaN,장,계획관리지역,지정되지않음,공업용,중로한면,경상남도 함안군 법수면,N,자동화재탐지설치대상,N,N,Y,N,N,N,N
59197,제1종근린생활시설,경량철골구조,상업용,답,농림지역,지정되지않음,답기타,맹지,경상남도 함안군 대산면,N,일반대상물,N,N,Y,N,N,N,N


# simple imputation
## sicf : simple imputation categorical feature
## sinf : simple imputation numeric feature

### categorical

In [42]:
df_train_sicf = df_train_2.select_dtypes("object").replace({np.NaN : "결측"})
df_val_sicf = df_val_2.select_dtypes("object").replace({np.NaN : "결측"})
print(df_train_sicf.isnull().sum().sum())
print(df_val_sicf.isnull().sum().sum())

0
0


### numeric
####Mean

In [89]:
from sklearn.impute import SimpleImputer


imp_mean = SimpleImputer(strategy='mean')
imp_mean.fit(df_train_2.select_dtypes("number"))
imputed_train_df = imp_mean.transform(df_train_2.select_dtypes("number"))
imputed_train_df

array([[1.00000e+00, 3.00000e+00, 6.94200e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 4.62900e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 5.83800e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [0.00000e+00, 2.00000e+00, 2.43682e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 2.00000e+00, 2.83200e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 1.69260e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [0]:
df_train_sinf_1 = pd.DataFrame(imputed_train_df, columns=df_train_2.select_dtypes("number").columns)

In [91]:
imp_mean = SimpleImputer(strategy='mean')
imp_mean.fit(df_val_2.select_dtypes("number"))
imputed_val_df = imp_mean.transform(df_val_2.select_dtypes("number"))
imputed_val_df

array([[2.00000e+00, 7.71300e+01, 7.71300e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.05430e+02, 9.85900e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.17800e+02, 2.90430e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [1.00000e+00, 1.15500e+02, 2.90400e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.39050e+02, 3.78840e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 5.09320e+02, 2.66972e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [0]:
df_val_sinf_1 = pd.DataFrame(imputed_val_df, columns=df_val_2.select_dtypes("number").columns)

####Median

In [96]:
from sklearn.impute import SimpleImputer


imp_mean = SimpleImputer(strategy='median')
imp_mean.fit(df_train_2.select_dtypes("number"))
imputed_train_df = imp_mean.transform(df_train_2.select_dtypes("number"))
imputed_train_df

array([[1.00000e+00, 3.00000e+00, 6.94200e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 4.62900e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 5.83800e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [0.00000e+00, 2.00000e+00, 2.43682e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 2.00000e+00, 2.83200e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 1.69260e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [0]:
df_train_sinf_2 = pd.DataFrame(imputed_train_df, columns=df_train_2.select_dtypes("number").columns)

In [0]:
imp_mean = SimpleImputer(strategy='median')
imp_mean.fit(df_val_2.select_dtypes("number"))
imputed_val_df = imp_mean.transform(df_val_2.select_dtypes("number"))
imputed_val_df
df_val_sinf_2 = pd.DataFrame(imputed_val_df, columns=df_val_2.select_dtypes("number").columns)

### frequent

In [102]:
imp_mean = SimpleImputer(strategy='most_frequent')
imp_mean.fit(df_train_2.select_dtypes("number"))
imputed_train_df = imp_mean.transform(df_train_2.select_dtypes("number"))
imputed_train_df

array([[1.00000e+00, 3.00000e+00, 6.94200e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 4.62900e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 5.83800e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [0.00000e+00, 2.00000e+00, 2.43682e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 2.00000e+00, 2.83200e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 1.69260e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [0]:
df_train_sinf_3 = pd.DataFrame(imputed_train_df, columns=df_train_2.select_dtypes("number").columns)

In [0]:
imp_mean = SimpleImputer(strategy='most_frequent')
imp_mean.fit(df_val_2.select_dtypes("number"))
imputed_val_df = imp_mean.transform(df_val_2.select_dtypes("number"))
imputed_val_df
df_val_sinf_3 = pd.DataFrame(imputed_val_df, columns=df_val_2.select_dtypes("number").columns)

### constant

In [108]:
imp_mean = SimpleImputer(strategy='constant')
imp_mean.fit(df_train_2.select_dtypes("number"))
imputed_train_df = imp_mean.transform(df_train_2.select_dtypes("number"))
imputed_train_df

array([[1.00000e+00, 3.00000e+00, 6.94200e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 4.62900e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 5.83800e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [0.00000e+00, 2.00000e+00, 2.43682e+03, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 2.00000e+00, 2.83200e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 3.00000e+00, 1.69260e+02, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [0]:
df_train_sinf_4 = pd.DataFrame(imputed_train_df, columns=df_train_2.select_dtypes("number").columns)

In [0]:
imp_mean = SimpleImputer(strategy='constant')
imp_mean.fit(df_val_2.select_dtypes("number"))
imputed_val_df = imp_mean.transform(df_val_2.select_dtypes("number"))
imputed_val_df
df_val_sinf_4 = pd.DataFrame(imputed_val_df, columns=df_val_2.select_dtypes("number").columns)

####KNN

In [18]:
!pip install impyute

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000)


imputed_training=fast_knn(df_train_2.select_dtypes("number").values, k=3)

In [0]:
df_train_sinf_5 = pd.DataFrame(imputed_training, columns=df_train_2.select_dtypes("number").columns)

In [0]:
df_train_sinf_5

,fr_yn,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,e_14,e_15,e_16,e_17,e_18,e_t,g_14,g_15,g_16,g_17,g_18,g_t
0,1.0,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.00000,283.85,1.722208,0.9,200.0,96.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,17360.0,4547.0,6.397596e+05,133.0,137.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.00000,292.65,1.722208,0.5,20.0,74.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,1791.0,6388.0,1.225810e+05,489.0,85.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.00000,288.65,1.722208,2.0,90.0,21.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,17285.0,3340.0,6.181050e+05,143.0,137.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.00000,293.75,1.722208,0.4,0.0,91.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,7327.0,179.0,7.195420e+05,1585.0,176.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,0.000,0.0000,0.0,1960.865319,1.000000,0.00000,285.85,1.722208,0.6,360.0,89.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,17278.0,4822.0,6.397596e+05,603.0,137.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,0.0,2.0,423.000,423.0000,3010.0,1996.000000,2.003621,0.04126,284.35,1.722208,0.2,0.0,41.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,20955.0,3435.0,6.397596e+05,895.0,139.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59195,0.0,35.0,5593.711,54835.0382,0.0,1984.654245,171.000000,0.00000,287.95,1.722208,1.9,20.0,72.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,21014.0,1137.0,1.849455e+06,88.0,139.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59196,0.0,2.0,2436.820,2468.8000,6512.0,2002.829729,4.000000,0.00000,286.45,1.722208,3.2,230.0,58.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,3014.0,5875.0,6.397596e+05,25.0,139.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59197,0.0,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.00000,291.35,1.000000,1.4,360.0,95.0,3.465472,3.132913,3.528796,3.456091,3.316866,1.69283,3.710709,1.653641,0.407825,0.392645,2.93005,2.93005,2.706786,2.778253,3753.0,8579.0,6.397596e+05,510.0,139.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0,0.0,0.0,3869.0,6348.0,8084.0,20008.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
df_train_sinf_5.to_csv(path + "imputed_knn.csv")

In [0]:
df_train_sinf_5 = pd.read_csv(path+"imputed_knn.csv")

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000)


imputed_training=fast_knn(df_val_2.select_dtypes("number").values, k=3)

In [0]:
df_val_sinf_5 = pd.DataFrame(imputed_training, columns=df_val_2.select_dtypes("number").columns)

# 범주변수 수정 결과 극혐

In [127]:
df_train_2, df_val_2, df_test_2 = preprocessing()
df_train_2[df_train_sicf.columns] = df_train_sicf
df_val_2[df_val_sicf.columns] = df_val_sicf
X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
f1_rfc(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):

100%|██████████| 10/10 [04:26<00:00, 26.68s/it]


0.508986928104575

# mean결과

In [128]:
df_train_2, df_val_2, df_test_2 = preprocessing()
df_train_2[df_train_sinf_1.columns] = df_train_sinf_1
df_val_2[df_val_sinf_1.columns] = df_val_sinf_1
X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
f1_rfc(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):

100%|██████████| 10/10 [04:36<00:00, 27.57s/it]


0.5108958837772396

#median 결과

In [129]:
df_train_2, df_val_2, df_test_2 = preprocessing()
df_train_2[df_train_sinf_2.columns] = df_train_sinf_2
df_val_2[df_val_sinf_2.columns] = df_val_sinf_2
X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
f1_rfc(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):

100%|██████████| 10/10 [04:38<00:00, 27.88s/it]


0.5089463220675946

# frequent

In [130]:
df_train_2, df_val_2, df_test_2 = preprocessing()
df_train_2[df_train_sinf_3.columns] = df_train_sinf_3
df_val_2[df_val_sinf_3.columns] = df_val_sinf_3
X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
f1_rfc(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):

100%|██████████| 10/10 [04:38<00:00, 27.69s/it]


0.505966587112172

# constant

In [131]:
df_train_2, df_val_2, df_test_2 = preprocessing()
df_train_2[df_train_sinf_3.columns] = df_train_sinf_4
df_val_2[df_val_sinf_3.columns] = df_val_sinf_4
X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
f1_rfc(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):

100%|██████████| 10/10 [04:23<00:00, 26.27s/it]


0.5002036659877802

# KNN

In [133]:
df_train_2, df_val_2, df_test_2 = preprocessing()
df_train_2[df_train_sinf_5.columns] = df_train_sinf_5
df_val_2[df_val_sinf_5.columns] = df_val_sinf_5
X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
f1_rfc(X_train, y_train, X_val, y_val)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


100%|██████████| 10/10 [05:05<00:00, 30.17s/it]


0.506923076923077